# ETL de los dataset de YElP

In [2]:
#inportamos librerias necesarias
import pandas as pd

In [3]:
#instalamos pyarrow para el manejo de archivos parquet
#%pip install pyarrow


                                              0.0/21.5 MB ? eta -:--:--
                                              0.1/21.5 MB 1.1 MB/s eta 0:00:20
                                              0.2/21.5 MB 2.3 MB/s eta 0:00:10
     -                                        0.6/21.5 MB 4.0 MB/s eta 0:00:06
     --                                       1.1/21.5 MB 6.0 MB/s eta 0:00:04
     ---                                      1.8/21.5 MB 8.0 MB/s eta 0:00:03
     ----                                     2.4/21.5 MB 9.1 MB/s eta 0:00:03
     -----                                    3.1/21.5 MB 9.9 MB/s eta 0:00:02
     ------                                   3.7/21.5 MB 10.4 MB/s eta 0:00:02
     --------                                 4.4/21.5 MB 10.7 MB/s eta 0:00:02
     ---------                                5.0/21.5 MB 11.0 MB/s eta 0:00:02
     ----------                               5.7/21.5 MB 11.4 MB/s eta 0:00:02
     -----------                              6.3/21.5 


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: C:\Users\andre\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## Businees

In [3]:
#cargamos dataset de businees
business= pd.read_json('Dataset\yelp_academic_dataset_business.json', lines = True)

In [4]:
#Verificamos
business.head(1)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None


In [5]:
#Comprovamos dimenciones
business.shape

(150346, 14)

In [6]:
#informaicon basica del DF
business.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150346 entries, 0 to 150345
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   business_id   150346 non-null  object 
 1   name          150346 non-null  object 
 2   address       150346 non-null  object 
 3   city          150346 non-null  object 
 4   state         150346 non-null  object 
 5   postal_code   150346 non-null  object 
 6   latitude      150346 non-null  float64
 7   longitude     150346 non-null  float64
 8   stars         150346 non-null  float64
 9   review_count  150346 non-null  int64  
 10  is_open       150346 non-null  int64  
 11  attributes    136602 non-null  object 
 12  categories    150243 non-null  object 
 13  hours         127123 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 16.1+ MB


In [7]:
#identifiacion de datos vacios o nulos
business.isna().sum()

business_id         0
name                0
address             0
city                0
state               0
postal_code         0
latitude            0
longitude           0
stars               0
review_count        0
is_open             0
attributes      13744
categories        103
hours           23223
dtype: int64

como se mensiono en la prupuesta, los estados que utilizaremos son los siguientes:

    California: CA
    Nueva York: NY
    Florida: FL
    Nevada: NV
    Illinois: IL
asi que filtraremos la data para quedarnos solo con informacion de esos estados.

In [8]:
#filtraremos el DF, primero cetiamos los estados a utilziar 
estados_validos = ['CA', 'NY', 'FL', 'NV', 'IL']
# Filtrar los registros con las claves de estados válidos
business_filtrado = business[business['state'].isin(estados_validos)]

verificamos cambios, con una visualziacion de .head() y .shape

In [9]:
business_filtrado.head(3)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
7,qkRM_2X51Yqxk3btlwAQIg,Temple Beth-El,400 Pasadena Ave S,St. Petersburg,FL,33707,27.766590,-82.732983,3.5,5,1,None,"Synagogues, Religious Organizations","{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ..."
10,UJsufbvfyfONHeWdvAHKjA,Marshalls,21705 Village Lakes Sc Dr,Land O' Lakes,FL,34639,28.190459,-82.457380,3.5,6,1,"{'RestaurantsPriceRange2': '2', 'BikeParking':...","Department Stores, Shopping, Fashion","{'Monday': '9:30-21:30', 'Tuesday': '9:30-21:3..."


In [10]:
business_filtrado.shape

(41393, 14)

ahora filtramos los negocios solo con aquellos que pertenecen a la categoria de hotel, nightlife o bars

In [11]:
# Limpiar los valores faltantes en la columna "categories"
business_filtrado['categories'] = business_filtrado['categories'].fillna('')

# Filtrar las filas donde la columna "categories" contiene las palabras "hotel", "nightlife" o "bars"
business_filtrado_completo = business_filtrado[
    (business_filtrado['categories'].str.lower().str.strip().str.contains('hotel')) |
    (business_filtrado['categories'].str.lower().str.strip().str.contains('nightlife')) |
    (business_filtrado['categories'].str.lower().str.strip().str.contains('bars'))
]

C:\Users\andre\AppData\Local\Temp\ipykernel_14356\446031116.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  business_filtrado['categories'] = business_filtrado['categories'].fillna('')


verificamos cambios, con una visualziacion de .head() y .shape

In [12]:
business_filtrado_completo.head(3)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
39,fSCNwMtNNQY9QT69Cj9fiA,Sierra Pro Events,,Sparks,NV,89431,39.540154,-119.748395,5.0,7,1,"{'BusinessAcceptsCreditCards': 'True', 'WiFi':...","Musicians, DJs, Karaoke, Event Planning & Serv...","{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ..."
79,pJfh3Ct8iL58NZa8ta-a5w,Top Shelf Sports Lounge,3173 Cypress Ridge Blvd,Wesley Chapel,FL,33544,28.196252,-82.380615,4.5,95,1,"{'BestNights': '{'monday': False, 'tuesday': F...","Burgers, Sports Bars, Bars, Lounges, Restauran...","{'Monday': '11:30-22:0', 'Tuesday': '11:30-23:..."
141,SZU9c8V2GuREDN5KgyHFJw,Santa Barbara Shellfish Company,230 Stearns Wharf,Santa Barbara,CA,93101,34.408715,-119.685019,4.0,2404,1,"{'OutdoorSeating': 'True', 'RestaurantsAttire'...","Live/Raw Food, Restaurants, Seafood, Beer Bar,...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-21:0', ..."


In [13]:
business_filtrado_completo.shape

(5883, 14)

se guardaran todos los cambios en un dataset llamado business_filtrado_completo en formato parquet en la carpeta Dataset_filtrados

In [14]:
#Guardemos el dataset de business filtrado
#business_filtrado_completo.to_parquet(r'Dataset_filtrados\business_filtrado_completo')

## Review

con el data set de business filtrado tanto por estados como por nichos, se utilizaran los datos de la columna business_id para filtrar los dataset de review

debido a que los dataset tienen mucho peso, la lectura y manejo de dichos datos se hará en bloques

cargamos los dataset del 0 al 11

In [13]:
#cargamos los dataset del 0 al 11
review0 = pd.read_parquet(r'Dataset\review\0.parquet')
review1 = pd.read_parquet(r'Dataset\review\1.parquet')
review2 = pd.read_parquet(r'Dataset\review\2.parquet')
review3 = pd.read_parquet(r'Dataset\review\3.parquet')
review4 = pd.read_parquet(r'Dataset\review\4.parquet')
review5 = pd.read_parquet(r'Dataset\review\5.parquet')
review6 = pd.read_parquet(r'Dataset\review\6.parquet')
review7 = pd.read_parquet(r'Dataset\review\7.parquet')
review8 = pd.read_parquet(r'Dataset\review\8.parquet')
review9 = pd.read_parquet(r'Dataset\review\9.parquet')
review10 = pd.read_parquet(r'Dataset\review\10.parquet')
review11 = pd.read_parquet(r'Dataset\review\11.parquet')


In [14]:
#se hace solo la revision de uno de los dataset, ya que estos son particiones de uno mas grande,
# por lo que las columnas en los demas dataset son iguales
review0.head(1)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11


In [15]:
# Filtrar los valores únicos de business_id del DataFrame business_filtrado_completo
business_ids_filtrados = business_filtrado_completo['business_id'].unique()

# Filtrar el DataFrame review0 para que solo contenga los business_id filtrados
review0_filtrado = review0[review0['business_id'].isin(business_ids_filtrados)]
review1_filtrado = review1[review1['business_id'].isin(business_ids_filtrados)]
review2_filtrado = review2[review2['business_id'].isin(business_ids_filtrados)]
review3_filtrado = review3[review3['business_id'].isin(business_ids_filtrados)]
review4_filtrado = review4[review4['business_id'].isin(business_ids_filtrados)]
review5_filtrado = review5[review5['business_id'].isin(business_ids_filtrados)]
review6_filtrado = review6[review6['business_id'].isin(business_ids_filtrados)]
review7_filtrado = review7[review7['business_id'].isin(business_ids_filtrados)]
review8_filtrado = review8[review8['business_id'].isin(business_ids_filtrados)]
review9_filtrado = review9[review9['business_id'].isin(business_ids_filtrados)]
review10_filtrado = review10[review10['business_id'].isin(business_ids_filtrados)]
review11_filtrado = review11[review11['business_id'].isin(business_ids_filtrados)]



In [17]:
#verificamos dimenciones
print(review0_filtrado.shape,
      review1_filtrado.shape,
      review2_filtrado.shape,
      review3_filtrado.shape,
      review4_filtrado.shape,
      review5_filtrado.shape,
      review6_filtrado.shape )

(23417, 9) (24011, 9) (23718, 9) (22250, 9) (22840, 9) (21762, 9) (21248, 9)


podemos observar que pasamos de tener 250mil datos en cada dataset a tener entre 24mil y 22 mil en cada uno

In [18]:
# convinemos los DF 
reviews = [review0_filtrado, review1_filtrado, review2_filtrado, review3_filtrado, review4_filtrado, review5_filtrado, review6_filtrado, review7_filtrado, review8_filtrado, review9_filtrado, review10_filtrado, review11_filtrado]
# Combinar los DataFrames en uno solo
Total_reviews1 = pd.concat(reviews, ignore_index=True)



El resultado de este bloque de archivos se guardara en un parquet llamado Total_reviews1

In [19]:
#guardamos el dataset de reviews filtrado
#Total_reviews1.to_parquet(r'Dataset_filtrados\Total_reviews1')

cargamos los dataset del 12 al 21


In [2]:
#cargamos los dataset del 12 al 21
review12 = pd.read_parquet(r'Dataset\review\12.parquet')
review13 = pd.read_parquet(r'Dataset\review\13.parquet')
review14 = pd.read_parquet(r'Dataset\review\14.parquet')
review15 = pd.read_parquet(r'Dataset\review\15.parquet')
review16 = pd.read_parquet(r'Dataset\review\16.parquet')
review17 = pd.read_parquet(r'Dataset\review\17.parquet')
review18 = pd.read_parquet(r'Dataset\review\18.parquet')
review19 = pd.read_parquet(r'Dataset\review\19.parquet')
review20 = pd.read_parquet(r'Dataset\review\20.parquet')
review21 = pd.read_parquet(r'Dataset\review\21.parquet')


In [14]:
# Filtrar los valores únicos de business_id del DataFrame business_filtrado_completo
business_ids_filtrados = business_filtrado_completo['business_id'].unique()

# Filtrar el DataFrame review12 para que solo contenga los business_id filtrados
review12_filtrado = review12[review12['business_id'].isin(business_ids_filtrados)]
review13_filtrado = review13[review13['business_id'].isin(business_ids_filtrados)]
review14_filtrado = review14[review14['business_id'].isin(business_ids_filtrados)]
review15_filtrado = review15[review15['business_id'].isin(business_ids_filtrados)]
review16_filtrado = review16[review16['business_id'].isin(business_ids_filtrados)]
review17_filtrado = review17[review17['business_id'].isin(business_ids_filtrados)]
review18_filtrado = review18[review18['business_id'].isin(business_ids_filtrados)]
review19_filtrado = review19[review19['business_id'].isin(business_ids_filtrados)]
review20_filtrado = review20[review20['business_id'].isin(business_ids_filtrados)]
review21_filtrado = review21[review21['business_id'].isin(business_ids_filtrados)]

In [15]:
#verificamos dimenciones
print(review12_filtrado.shape,
      review13_filtrado.shape,
      review21_filtrado.shape )

(21756, 9) (21444, 9) (20392, 9)


In [16]:
# convinemos los DF 
reviews_2 = [review12_filtrado, review13_filtrado, review14_filtrado, review15_filtrado, review16_filtrado, review17_filtrado, review18_filtrado, review19_filtrado, review20_filtrado, review21_filtrado]

# Combinar los DataFrames en uno solo
Total_reviews2 = pd.concat(reviews_2, ignore_index=True)

El resultado de este bloque de archivos se guardara en un parquet llamado Total_reviews2

In [17]:
#guardamos el dataset de reviews filtrado
#Total_reviews2.to_parquet(r'Dataset_filtrados\Total_reviews2')

cargamos los dataset del 22 al 27


In [13]:
#cargamos los dataset del 22 al 27
review22 = pd.read_parquet(r'Dataset\review\22.parquet')
review23 = pd.read_parquet(r'Dataset\review\23.parquet')
review24 = pd.read_parquet(r'Dataset\review\24.parquet')
review25 = pd.read_parquet(r'Dataset\review\25.parquet')
review26 = pd.read_parquet(r'Dataset\review\26.parquet')
review27 = pd.read_parquet(r'Dataset\review\27.parquet')

In [14]:
# Filtrar los valores únicos de business_id del DataFrame business_filtrado_completo
business_ids_filtrados = business_filtrado_completo['business_id'].unique()

# Filtrar el DataFrame review12 para que solo contenga los business_id filtrados
review22_filtrado = review22[review22['business_id'].isin(business_ids_filtrados)]
review23_filtrado = review23[review23['business_id'].isin(business_ids_filtrados)]
review24_filtrado = review24[review24['business_id'].isin(business_ids_filtrados)]
review25_filtrado = review25[review25['business_id'].isin(business_ids_filtrados)]
review26_filtrado = review26[review26['business_id'].isin(business_ids_filtrados)]
review27_filtrado = review27[review27['business_id'].isin(business_ids_filtrados)]


In [15]:
#verificamos dimenciones
print(review22_filtrado.shape,
      review24_filtrado.shape,
      review27_filtrado.shape )

(22960, 9) (24387, 9) (22602, 9)


In [16]:
# convinemos los DF 
reviews_3 = [review22_filtrado, review23_filtrado, review24_filtrado, review25_filtrado, review26_filtrado, review27_filtrado]

# Combinar los DataFrames en uno solo
Total_reviews3 = pd.concat(reviews_3, ignore_index=True)

El resultado de este bloque de archivos se guardara en un parquet llamado Total_reviews3

In [17]:
#guardamos el dataset de reviews filtrado
#Total_reviews3.to_parquet(r'Dataset_filtrados\Total_reviews3')

Debido al tamaño de los archivos que se manejan, es necesario reiniciar el kernel. Por lo tanto, a continuación, se cargan los archivos resultantes de cada uno de los bloques de código previamente trabajados, con el propósito de unificarlos en un solo archivo Parquet y realizar una limpieza profunda de los datos.

In [2]:
# se cargan los dataset
Total_reviews = pd.read_parquet(r'Dataset_filtrados\Total_reviews')
Total_reviews2 = pd.read_parquet(r'Dataset_filtrados\Total_reviews2')
Total_reviews3 = pd.read_parquet(r'Dataset_filtrados\Total_reviews3')


In [8]:
#se seleccionana los DF
dataframes = [Total_reviews, Total_reviews2, Total_reviews3]

# Unir los DataFrames en uno solo
Total_reviews = pd.concat(dataframes, ignore_index=True)

In [9]:
#verificamos dimenciones
Total_reviews.shape

(617020, 9)

El resultado de este bloque de archivos se guardara en un parquet llamado Total_reviews

In [10]:
#guardamos el dataset de reviews filtrado
#Total_reviews.to_parquet(r'Dataset_filtrados\Total_reviews')

Con el archivo que contiene el total de reseñas unificadas y filtradas, es posible eliminar los archivos utilizados para crear dicho conjunto, ya que este archivo concentra toda la información de dichas reseñas.

In [3]:
#ahora haremos etl para el archivo de Total_reviews
Total_reviews = pd.read_parquet(r'Dataset_filtrados\Total_reviews')

In [7]:
#verificamos con un head() para ver informacion general de las columnas
Total_reviews.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,pUycOfUwM8vqX7KjRRhUEA,59MxRhNVhU9MYndMkz0wtw,gebiRewfieSdtt17PTW6Zg,3.0,0,0,0,Had a party of 6 here for hibachi. Our waitres...,2016-07-25 07:31:06
1,eCiWBf1CJ0Zdv1uVarEhhw,OhECKhQEexFypOMY6kypRw,vC2qm1y3Au5czBtbhc-DNw,4.0,0,0,0,"Yes, this is the only sushi place in town. How...",2013-09-04 03:48:20
2,pHwbdway4yeI-dSSmZA7-Q,qEEk0PuoH1dVa619t8fgpw,PY9GRfzr4nTZeINf346QOw,4.0,0,0,0,We checked in around 2:30 pm. Check-in was qu...,2017-09-20 16:16:47
3,4zopEEPqfwm-c_FNpeHZYw,JYYYKt6TdVA4ng9lLcXt_g,SZU9c8V2GuREDN5KgyHFJw,5.0,0,0,0,We were a bit weary about trying the Shellfish...,2016-05-31 02:14:54
4,HuWEnZr7-0HveaqXxyywgg,Eb8LgZArKmW0KgW6lTni-w,YbnJYHNp_fHbI-hcFg48vQ,5.0,2,0,2,The kayaking tour at the Santa Cruz Island was...,2009-08-03 21:55:27


In [11]:
#dado a que el archivo hace referencia a todas las reseñas de un negocio, se verifica que ciertamente haya varias reseñas de un negocio
Total_reviews.loc[Total_reviews['business_id'] == 'gebiRewfieSdtt17PTW6Zg', :]

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,pUycOfUwM8vqX7KjRRhUEA,59MxRhNVhU9MYndMkz0wtw,gebiRewfieSdtt17PTW6Zg,3.0,0,0,0,Had a party of 6 here for hibachi. Our waitres...,2016-07-25 07:31:06
104,_Ub20uO1MKy4XOVPOdzpqw,lUYboGI6aFbZ0dX27pijpA,gebiRewfieSdtt17PTW6Zg,1.0,0,2,0,"Sat down over a half hour, only miso and drink...",2017-06-28 01:04:59
146,-DjIfoNFAiT5J4kF9hXocQ,SrfDRvGKI8FQq9LCr0dQuQ,gebiRewfieSdtt17PTW6Zg,3.0,9,4,6,Not bad. Sushi roll was good but options were ...,2017-01-14 23:31:35
148,URe-2Zlv7RoY4Ycr9JYVWw,SfMf7DEX7xT82OCcbZFsgQ,gebiRewfieSdtt17PTW6Zg,5.0,1,0,1,I'm very picky when it comes to Sushi. Quality...,2016-07-14 23:09:38
418,tHP5ihPd_W9OzfkKEWdIwA,aH3AH7V9g60w4pt3LpJ1_Q,gebiRewfieSdtt17PTW6Zg,4.0,0,0,0,Even though the sushi masters are not actually...,2014-11-30 07:35:54
...,...,...,...,...,...,...,...,...,...
66873,nVkXTrVqDdOkun2vFHmGBQ,3cCuXEfKOdq72ZHKq3xKKg,gebiRewfieSdtt17PTW6Zg,3.0,0,0,0,Sushi and service are good. They put all of t...,2011-10-13 23:32:19
66882,eBTWD9S3rNhxcQa7Jq47KQ,xU4PTLbyrLrAmc9P6NdB9g,gebiRewfieSdtt17PTW6Zg,1.0,0,0,0,"We had a 730 reservation, didn't get seated un...",2016-03-29 18:22:41
66924,FDYhQ4IkZecRkfOJwaV4VA,S8F0sdoihQ6wK9zrMhaWDQ,gebiRewfieSdtt17PTW6Zg,5.0,0,0,0,I've been coming here since it was called Some...,2018-07-21 05:32:58
66955,KtTZM-9W4Lfpuq3-LOSBgg,deU2x7lqmIQ9hee_nqvswg,gebiRewfieSdtt17PTW6Zg,5.0,1,0,0,Had a great meal for my dads 55th! Staff was a...,2017-06-16 02:13:34


In [5]:
# observamos que el tipo de dato en las columanas concuerda con su contenido 
Total_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 617020 entries, 0 to 617019
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   review_id    617020 non-null  object 
 1   user_id      617020 non-null  object 
 2   business_id  617020 non-null  object 
 3   stars        617020 non-null  float64
 4   useful       617020 non-null  int64  
 5   funny        617020 non-null  int64  
 6   cool         617020 non-null  int64  
 7   text         617020 non-null  object 
 8   date         617020 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 42.4+ MB


In [6]:
# se evidencia que el dataset no posee datos nulos
Total_reviews.isna().sum()

review_id      0
user_id        0
business_id    0
stars          0
useful         0
funny          0
cool           0
text           0
date           0
dtype: int64

## USER

con el dataset de reviewa filtrado tanto por estados como por nichos, se utilizaran los datos de la columna user_id para filtrar los dataset de user

debido a que los dataset tienen mucho peso, la lectura y manejo de dichos datos se hará en bloques

cargamos los dataset del 0 al 3

In [3]:
#cargamos los dataset del 0 al 3
user0 = pd.read_parquet(r'Dataset\user\0.parquet')
user1 = pd.read_parquet(r'Dataset\user\1.parquet')
user2 = pd.read_parquet(r'Dataset\user\2.parquet')
user3 = pd.read_parquet(r'Dataset\user\3.parquet')


In [3]:
#verificamso los datos del df
user0.head(1)

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,...,65,55,56,18,232,844,467,467,239,180


In [8]:
# verificamos dimenciones de una pequeña muestra del los archivos
user0.shape

(250000, 22)

In [6]:
# Filtrar los valores únicos de user_id del DataFrame Total_reviews
user_ids_filtrados = Total_reviews['user_id'].unique()

# Filtrar el DataFrame user para que solo contenga los user_id filtrados
user0_filtrado = user0[user0['user_id'].isin(user_ids_filtrados)]
user1_filtrado = user1[user1['user_id'].isin(user_ids_filtrados)]
user2_filtrado = user2[user2['user_id'].isin(user_ids_filtrados)]
user3_filtrado = user3[user3['user_id'].isin(user_ids_filtrados)]


In [7]:
#verificamos dimenciones resultantes luego del filtrado
print(user0_filtrado.shape, user1_filtrado.shape,user2_filtrado.shape,user3_filtrado.shape,)

(58233, 22) (48500, 22) (40585, 22) (34975, 22)


In [10]:
# convinemos los DF 
user_1 = [user0_filtrado, user1_filtrado, user2_filtrado, user3_filtrado]

# Combinar los DataFrames en uno solo
Total_user1 = pd.concat(user_1, ignore_index=True)

In [11]:
#verificamos dimenciones
Total_user1.shape

(182293, 22)

El resultado de este bloque de archivos se guardara en un parquet llamado Total_user1

In [12]:
#guardamos el dataset de reviews filtrado
#Total_user1.to_parquet(r'Dataset_filtrados\Total_user1')

cargamos los dataset del 4 al 7

In [4]:
# cargamos los dataset del 4 al 7
user4 = pd.read_parquet(r'Dataset\user\4.parquet')
user5 = pd.read_parquet(r'Dataset\user\5.parquet')
user6 = pd.read_parquet(r'Dataset\user\6.parquet')
user7 = pd.read_parquet(r'Dataset\user\7.parquet')

In [5]:
# Filtrar los valores únicos de user_id del DataFrame Total_reviews
user_ids_filtrados = Total_reviews['user_id'].unique()

# Filtrar el DataFrame user para que solo contenga los user_id filtrados
user4_filtrado = user4[user4['user_id'].isin(user_ids_filtrados)]
user5_filtrado = user5[user5['user_id'].isin(user_ids_filtrados)]
user6_filtrado = user6[user6['user_id'].isin(user_ids_filtrados)]
user7_filtrado = user7[user7['user_id'].isin(user_ids_filtrados)]


In [6]:
#Verificamos dimenciones
print(user4_filtrado.shape, user5_filtrado.shape, user6_filtrado.shape, user7_filtrado.shape)


(33971, 22) (31340, 22) (28146, 22) (26212, 22)


In [7]:
# convinemos los DF 
user_2 = [user4_filtrado, user5_filtrado, user6_filtrado, user7_filtrado]

# Combinar los DataFrames en uno solo
Total_user2 = pd.concat(user_2, ignore_index=True)

In [8]:
#verificamos dimencion
Total_user2.shape

(119669, 22)

El resultado de este bloque de archivos se guardara en un parquet llamado Total_user2

In [9]:
#guardamos el dataset de reviews filtrado
# Total_user2.to_parquet(r'Dataset_filtrados\Total_user2')

Debido al tamaño de los archivos que se manejan, es necesario reiniciar el kernel. Por lo tanto, a continuación, se cargan los archivos resultantes de cada uno de los bloques de código previamente trabajados, con el propósito de unificarlos en un solo archivo Parquet y realizar una limpieza profunda de los datos.

In [2]:
#se cargan los dataset
Total_user1 = pd.read_parquet(r'Dataset_filtrados\Total_user1')
Total_user2 = pd.read_parquet(r'Dataset_filtrados\Total_user2')

In [3]:
#unir datasets
dataframes = [Total_user1, Total_user2]

# Unir los DataFrames en uno solo
Total_user = pd.concat(dataframes, ignore_index=True)

In [4]:
#verificamos dimenciones del resultado
Total_user.shape

(301962, 22)

El resultado de este bloque de archivos se guardara en un parquet llamado Total_reviews

In [5]:
#guardamos el dataset de reviews filtrado
Total_user.to_parquet(r'Dataset_filtrados\Total_user')

Con el archivo que contiene el total de user unificados y filtradas, es posible eliminar los archivos utilizados para crear dicho conjunto, ya que este archivo concentra toda la información de dichas reseñas.

In [3]:
#cargamos el archivo con el conglomerado de los users
Total_user= pd.read_parquet(r'Dataset_filtrados\Total_user')

In [5]:
#vemos los datos de forma general con el .head()
Total_user.head()

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,...,65,55,56,18,232,844,467,467,239,180
1,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,2008-07-25 10:41:00,2086,1010,1003,"2009,2010,2011,2012,2013","LuO3Bn4f3rlhyHIaNfTlnA, j9B4XdHUhDfTKVecyWQgyA...",52,...,13,10,17,3,66,96,119,119,35,18
2,q_QQ5kBBwlCcbL1s4NVK3g,Jane,1221,2005-03-14 20:26:35,14953,9940,11211,"2006,2007,2008,2009,2010,2011,2012,2013,2014","xBDpTUbai0DXrvxCe3X16Q, 7GPNBO496aecrjJfW6UWtg...",1357,...,163,191,361,147,1212,5696,2543,2543,815,323
3,xoZvMJPDW6Q9pDAXI0e_Ww,Ryan,535,2009-05-27 06:12:10,1130,487,573,"2009,2010,2011,2012","6tbXpUIU6upoeqWNDo9k_A, Vlab9b73R5qPLIv6tE4DJA...",31,...,9,1,3,5,31,41,36,36,24,7
4,vVukUtqoLF5BvH_VtQFNoA,Charlene,37,2011-01-29 17:18:59,63,3,27,,"zkK6c9BcDyqreU0fqI_JLQ, opI1hhhFqElB6pptNH9ZqA...",4,...,0,0,0,0,1,5,0,0,1,0


In [4]:
#verificamos dimenciones
Total_user.shape

(301962, 22)

In [7]:
#informacion general de los datos
Total_user.info()
#observamos que el tipo de dato en las columanas concuerda con su contenido 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301962 entries, 0 to 301961
Data columns (total 22 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   user_id             301962 non-null  object 
 1   name                301962 non-null  object 
 2   review_count        301962 non-null  int64  
 3   yelping_since       301962 non-null  object 
 4   useful              301962 non-null  int64  
 5   funny               301962 non-null  int64  
 6   cool                301962 non-null  int64  
 7   elite               301962 non-null  object 
 8   friends             301962 non-null  object 
 9   fans                301962 non-null  int64  
 10  average_stars       301962 non-null  float64
 11  compliment_hot      301962 non-null  int64  
 12  compliment_more     301962 non-null  int64  
 13  compliment_profile  301962 non-null  int64  
 14  compliment_cute     301962 non-null  int64  
 15  compliment_list     301962 non-nul

In [8]:
#verificacion de datos nulos
Total_user.isna().sum()
#el dataset no posee datos nulos

user_id               0
name                  0
review_count          0
yelping_since         0
useful                0
funny                 0
cool                  0
elite                 0
friends               0
fans                  0
average_stars         0
compliment_hot        0
compliment_more       0
compliment_profile    0
compliment_cute       0
compliment_list       0
compliment_note       0
compliment_plain      0
compliment_cool       0
compliment_funny      0
compliment_writer     0
compliment_photos     0
dtype: int64

## tips

In [9]:
#cargamos el dataset de tips
tips = pd.read_json('Dataset\yelp_academic_dataset_tip.json', lines= True)

In [10]:
# observamos de forma general los datos contenidos
tips.head()

,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,2017-06-27 23:05:38,0
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,2012-10-06 19:43:09,0


In [11]:
#verificamos dimenciones
tips.shape

(908915, 5)

In [13]:
Total_reviews=pd.read_parquet('Dataset_filtrados\Total_reviews')

In [14]:
# Filtrar los valores únicos de user_id del DataFrame Total_reviews
business_ids_filtrados = Total_reviews['business_id'].unique()

# Filtrar el DataFrame tips para que solo contenga los business_id filtrados
tips_filtrado1 = tips[tips['business_id'].isin(business_ids_filtrados)]

In [15]:
#verificamos dimenciones
tips_filtrado1.shape

(79661, 5)

In [16]:
# Filtrar los valores únicos de user_id del DataFrame Total_reviews
user_ids_filtrados = Total_reviews['user_id'].unique()

# Filtrar el DataFrame tips para que solo contenga los user_id filtrados
tips_filtrado_completo = tips_filtrado1[tips_filtrado1['user_id'].isin(user_ids_filtrados)]


In [17]:
#verificamos dimenciones
tips_filtrado_completo.shape

(66363, 5)

Con esto garantizamos que el dataset de tips solo contenga el ID de los negocios que seleccionamos y el ID de los usuarios ya identificados en el taller de user

In [19]:
# hagamos ETL enel DF de tips_filtrado_completo
#verifiquemos informacion con un .head()
tips_filtrado_completo.head()

,user_id,business_id,text,date,compliment_count
8,VL12EhEdT4OWqGq0nIqkzw,xODBZmX4EmlVvbqtKN7YKg,Tacos,2012-07-27 01:48:24,0
13,MlnuJ7T14CE0JDK2ZIOx5g,MDr7KLYSPkEonvGojNEMBw,Let's go Yankees!,2011-07-20 21:52:57,0
15,j2sEA3hiUcwHfq9Ml6M47g,EXYbKA1tocvOK_1tXxZXLA,Don't go for dinner. They close at 6. Really Y...,2011-10-13 03:15:15,0
22,phKNHTaokisQkZ7qQbxryA,pWuoHHu0LuVrD0HjadDygg,Crepes please!!!,2010-11-21 20:52:49,0
30,jDThlALkraoQLLBYHqY7FQ,9DJhhBqQSu-gTBwaqdhgpQ,Come early for the best service as they fill u...,2016-01-16 19:13:57,0


In [20]:
#verificamosinformacion general
tips_filtrado_completo.info()
#observamos que el tipo de dato en las columanas concuerda con su contenido 

<class 'pandas.core.frame.DataFrame'>
Index: 66363 entries, 8 to 908908
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   user_id           66363 non-null  object        
 1   business_id       66363 non-null  object        
 2   text              66363 non-null  object        
 3   date              66363 non-null  datetime64[ns]
 4   compliment_count  66363 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 3.0+ MB


In [21]:
#identifiacionde datos nulos.
tips_filtrado_completo.isna().sum()
#el dataset no posee datos nulos

user_id             0
business_id         0
text                0
date                0
compliment_count    0
dtype: int64

guardamos el DF en un nuevo archivo llamado Total_tips

In [22]:
tips_filtrado_completo.to_parquet(r'Dataset_filtrados\Total_tips')


## checkin

In [23]:
#cargamos el dataset
checkin = pd.read_json('Dataset\yelp_academic_dataset_checkin.json', lines = True)

In [24]:
#vemos como se conforma la informacion de manera genral
checkin.head()

,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020..."
1,--0iUa4sNDFiZFrAdIWhZQ,"2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011..."
2,--30_8IhuyMHbSOcNWd6DQ,"2013-06-14 23:29:17, 2014-08-13 23:20:22"
3,--7PUidqRWpRSpXebiyxTg,"2011-02-15 17:12:00, 2011-07-28 02:46:10, 2012..."
4,--7jw19RH9JKXgFohspgQw,"2014-04-21 20:42:11, 2014-04-28 21:04:46, 2014..."


In [25]:
#verificamos informacion
checkin.shape

(131930, 2)

In [26]:
# Filtrar los valores únicos de user_id del DataFrame Total_reviews
business_ids_filtrados = Total_reviews['business_id'].unique()

# Filtrar el DataFrame checkin para que solo contenga los business_id filtrados
Total_checkin = checkin[checkin['business_id'].isin(business_ids_filtrados)]

In [27]:
#verificamos dimenciones
Total_checkin.shape

(5670, 2)

In [28]:
#verificamosinformacion general
Total_checkin.info()
#observamos que el tipo de dato en las columanas concuerda con su contenido 

<class 'pandas.core.frame.DataFrame'>
Index: 5670 entries, 7 to 131913
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   business_id  5670 non-null   object
 1   date         5670 non-null   object
dtypes: object(2)
memory usage: 132.9+ KB


In [29]:
#identifiacionde datos nulos.
Total_checkin.isna().sum()
#el dataset no posee datos nulos

business_id    0
date           0
dtype: int64

guardamos el DF en un nuevo archivo llamado tips_checkin

In [30]:
Total_checkin.to_parquet(r'Dataset_filtrados\Total_checkin')


# Eliminar Columnas

Información a considerar para el siguiente parte de ETL.
- 1 se borrarán las columnas que no influyan en los KPI que se plantearon para este proyecto. 
- 2 debajo de cada conjunto de código de cada datase estará el diccionario explicativo del contenido de cada columna.
 


In [ ]:
import pandas as pd

In [ ]:
# Configurar para mostrar todas las columnas y filas en la salida
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

In [ ]:
# cargamos cada uno de los dataset
business = pd.read_parquet(r'Dataset_filtrados\business_filtrado_completo')
checkin = pd.read_parquet(r'Dataset_filtrados\Total_checkin')
review = pd.read_parquet(r'Dataset_filtrados\Total_reviews')
tips = pd.read_parquet(r'Dataset_filtrados\Total_tips')
user = pd.read_parquet(r'Dataset_filtrados\Total_user')


## business

In [ ]:
#verificamos informacion general
business.head(1)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
39,fSCNwMtNNQY9QT69Cj9fiA,Sierra Pro Events,,Sparks,NV,89431,39.540154,-119.748395,5.0,7,1,"{'AcceptsInsurance': None, 'AgesAllowed': None...","Musicians, DJs, Karaoke, Event Planning & Serv...","{'Friday': '9:0-17:0', 'Monday': '9:0-17:0', '..."


In [ ]:
#Visuaisamos columnas
business.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours'],
      dtype='object')

Considerando los kpis, se ha tomado la decisión de eliminar as columnas que no tendrían influencia en estos con el fin de aligerar el peso de los dataset y mejorar su almacenamiento.

Por lo tanto las columnas 'is_open', 'hours', 'attributes’, al no tener información que se utilizara en el sistema de recomendación  y en la medición de los kpis serán eliminadas. 


In [ ]:
columnas_a_eliminar = ['is_open', 'hours', 'attributes']
business.drop(columns=columnas_a_eliminar, inplace=True)


In [ ]:
#verificamos cambios 
business.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'categories'],
      dtype='object')

In [ ]:
#verificamos la existencia de datos nulos
business.isna().sum()

business_id     0
name            0
address         0
city            0
state           0
postal_code     0
latitude        0
longitude       0
stars           0
review_count    0
categories      0
dtype: int64

In [ ]:
#ratificamos que los estads que se utilizaran son 
# 'NV': Nevada
# 'FL': Florida
# 'CA': California
# 'IL': Illinois
business['state'].unique()

array(['NV', 'FL', 'CA', 'IL'], dtype=object)

In [ ]:
business.to_parquet(r'Dataset_filtrados\business_filtrado_completo')

### Diccionario de Bussines 

•	business_id: Un identificador único para cada negocio en la base de datos.

•	name: El nombre del negocio.

•	address: La dirección del negocio.

•	city: La ciudad donde se encuentra el negocio.

•	state: El estado o provincia donde se encuentra el negocio.

•	postal_code: El código postal del área donde se encuentra el negocio.

•	latitude: La latitud geográfica del negocio.

•	longitude: La longitud geográfica del negocio.

•	stars: La calificación promedio del negocio en Yelp, en una escala de 1 a 5 estrellas.

•	review_count: El número total de reseñas que ha recibido el negocio en Yelp.

•	categories: Las categorías o etiquetas que describen el tipo de negocio, separadas por comas.


## checkin

In [ ]:
#visuaizamos datos de forma general
checkin.head(1)

,business_id,date
7,--ARBQr1WMsTWiwOKOj-FQ,"2014-12-12 00:44:23, 2015-01-09 00:19:52, 2015..."


In [ ]:
#verificamos la existencia de datos nulos
checkin.isna().sum()

business_id    0
date           0
dtype: int64

### Diccionario de checkin 

•	business_id: Es un identificador único para cada negocio en Yelp.

•	date: Representa las fechas en las que se registraron los check-ins para el negocio. Puede haber múltiples fechas separadas por comas para un mismo negocio, lo que sugiere múltiples check-ins en diferentes momentos.


In [ ]:
checkin.to_parquet(r'Dataset_filtrados\Total_checkin')


## union de business y checkin

Se ha pensado una nueva forma de abordar el área de negocio, a partir de este dataset, tomando en consideración las visitas a su página web y que hoy en día gran parte del mercadeos se da mediante internet, utilizaremos la columna date  para ver las visitas de un citio y según el promedio de las visitas de los negocios parecidos recomendar que se puede hacer desde el área de marketing

In [ ]:
# unimos los dataset de business y checkin para relacional las visitas a cada negocio 
Visitas = pd.merge(business, checkin, on='business_id')

In [ ]:
#visuaizamos datos de forma general
Visitas.head(1)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,categories,date
0,pJfh3Ct8iL58NZa8ta-a5w,Top Shelf Sports Lounge,3173 Cypress Ridge Blvd,Wesley Chapel,FL,33544,28.196252,-82.380615,4.5,95,"Burgers, Sports Bars, Bars, Lounges, Restauran...","2018-02-09 22:40:05, 2018-03-25 21:01:21, 2018..."


In [ ]:
# A partir de la columna date, que como se menciono anteriormente contiene fechas de las visitas en la pagina web de cada negocio,
# se genera una nueva columna que contiene la cantidad de visitas. 
Visitas['num_Visitas'] = Visitas['date'].apply(lambda x: len(x.split(',')))

In [ ]:
#verificamos datos.
Visitas.head(1)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,categories,date,num_Visitas
0,pJfh3Ct8iL58NZa8ta-a5w,Top Shelf Sports Lounge,3173 Cypress Ridge Blvd,Wesley Chapel,FL,33544,28.196252,-82.380615,4.5,95,"Burgers, Sports Bars, Bars, Lounges, Restauran...","2018-02-09 22:40:05, 2018-03-25 21:01:21, 2018...",111


A continuación, se calculará el promedio de visitas de cada nicho o negocio (hoteles, bars, nightlife) y para ello se crea la siguiente función

In [ ]:
def asignar_promedio(row):
    #promedio de visitas para hoteles, Aquí se calcula el promedio de visitas de todas los negocios en los cuales la columna categoría contenga la palabra ‘hotel’ 
    promedio_hotel= Visitas[Visitas['categories'].str.lower().str.strip().str.contains('hotel')]['num_Visitas'].mean() 
    #promedio de visitas para nightlife, Aquí se calcula el promedio de visitas de todas los negocios en los cuales la columna categoría contenga la palabra ‘nightlife’ 
    promedio_nightlife = Visitas[Visitas['categories'].str.lower().str.strip().str.contains('nightlife')]['num_Visitas'].mean()
    #promedio de visitas para bars, Aquí se calcula el promedio de visitas de todas los negocios en los cuales la columna categoría contenga la palabra ‘bars’ 
    promedio_bars= Visitas[Visitas['categories'].str.lower().str.strip().str.contains('bars')]['num_Visitas'].mean()
    if 'hotel' in row['categories'].lower():
        return promedio_hotel
    elif 'nightlife' in row['categories'].lower():
        return promedio_nightlife
    elif 'bars' in row['categories'].lower():
        return promedio_bars
    else:
        return None  # O el valor por defecto que desees

In [ ]:
#se crea una nueva columna aplicando la función previamente creada
Visitas['promedio_visitas'] = Visitas.apply(asignar_promedio, axis=1)


In [ ]:
Visitas.head(1)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,categories,date,num_Visitas,promedio_visitas
0,pJfh3Ct8iL58NZa8ta-a5w,Top Shelf Sports Lounge,3173 Cypress Ridge Blvd,Wesley Chapel,FL,33544,28.196252,-82.380615,4.5,95,"Burgers, Sports Bars, Bars, Lounges, Restauran...","2018-02-09 22:40:05, 2018-03-25 21:01:21, 2018...",111,258.313062


In [ ]:
# Función para asignar mensajes en función de si el valor es menor o mayor que el promedio
def resultado(row):
    if row['num_Visitas'] < row['promedio_visitas']:
        return 'menor al promedio'
    elif row['num_Visitas'] > row['promedio_visitas']:
        return 'mayor al promedio'
    else:
        return 'igual al promedio'


In [ ]:

# Aplicar la función a cada fila y crear una nueva columna 'mensaje_promedio'
Visitas['mensaje_promedio'] = Visitas.apply(resultado, axis=1)


In [ ]:
Visitas.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,categories,date,num_Visitas,promedio_visitas,mensaje_promedio
0,pJfh3Ct8iL58NZa8ta-a5w,Top Shelf Sports Lounge,3173 Cypress Ridge Blvd,Wesley Chapel,FL,33544,28.196252,-82.380615,4.5,95,"Burgers, Sports Bars, Bars, Lounges, Restauran...","2018-02-09 22:40:05, 2018-03-25 21:01:21, 2018...",111,258.313062,menor al promedio
1,SZU9c8V2GuREDN5KgyHFJw,Santa Barbara Shellfish Company,230 Stearns Wharf,Santa Barbara,CA,93101,34.408715,-119.685019,4.0,2404,"Live/Raw Food, Restaurants, Seafood, Beer Bar,...","2010-03-07 05:23:49, 2010-03-21 05:07:50, 2010...",6148,258.313062,mayor al promedio
2,7Du9oW73YcYFmXdtU5aiSg,Kinjo's Japanese Restaurant,2875 Tyrone Blvd N,Saint Petersburg,FL,33710,27.797263,-82.733913,3.5,17,"Japanese, Sushi Bars, Restaurants","2010-07-14 22:36:50, 2010-07-15 21:04:21, 2010...",31,253.113694,menor al promedio
3,znL4YrNbA2uwOQ3CchkmEA,Seaview Condominiums,14700 Gulf Blvd,Madeira Beach,FL,33708,27.798503,-82.798750,4.5,8,"Home Services, Vacation Rentals, Real Estate, ...","2015-06-14 01:25:52, 2015-06-17 21:28:10, 2015...",3,152.773946,menor al promedio
4,ZU6NodDOWaabGkeNpOWdXw,530 Pub & Grill,530 Cleveland St,Clearwater,FL,33755,27.965747,-82.798901,4.5,6,"American (Traditional), Restaurants, Burgers, ...",2021-07-30 17:28:16,1,258.313062,menor al promedio


## review

In [ ]:
#vemos datos en general
review.head(1)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,pUycOfUwM8vqX7KjRRhUEA,59MxRhNVhU9MYndMkz0wtw,gebiRewfieSdtt17PTW6Zg,3.0,0,0,0,Had a party of 6 here for hibachi. Our waitres...,2016-07-25 07:31:06


In [ ]:
#visualizamos columnas
review.columns

Index(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny',
       'cool', 'text', 'date'],
      dtype='object')

In [ ]:
# identificamos datos nulos
review.isna().sum()

review_id      0
user_id        0
business_id    0
stars          0
useful         0
funny          0
cool           0
text           0
date           0
dtype: int64

Las columnas a eliminar son ‘review_id’ dado que las claves (ID) de bussines y usser serán suficientes para asociar las review a cada negocio y usuario 

In [ ]:
columnas = ['review_id']
review.drop(columns=columnas, inplace=True)

In [ ]:
#verificamos resultados
review.head(1)

,user_id,business_id,stars,useful,funny,cool,text,date
0,59MxRhNVhU9MYndMkz0wtw,gebiRewfieSdtt17PTW6Zg,3.0,0,0,0,Had a party of 6 here for hibachi. Our waitres...,2016-07-25 07:31:06


### Diccionario de review 

•	user_id (cadena): Es el identificador único del usuario que ha escrito la reseña. Cada usuario tiene un id_usuario único que lo distingue de otros usuarios.

•	business_id (cadena): Es el identificador único del negocio que recibió la reseña. Cada negocio tiene una Identificación del negocio única que lo distingue de otros negocios.

•	stars (entero): Es la calificación de la reseña en términos de estrellas. Puede ser un valor entero entre 1 y 5, donde 1 es la calificación más baja y 5 es la calificación más alta.

•	useful (entero): Representa el número de votos útiles que ha recibido la reseña de otros usuarios. Los usuarios pueden votar si una reseña les resultó útil o no.

•	funny (entero): Representa el número de votos de otros usuarios que encontraron la reseña divertida. Los usuarios pueden votar si una reseña les resultó divertida o no.

•	cool (entero): Representa el número de votos de otros usuarios que encontraron la reseña fría o poco útil. Los usuarios pueden votar si una reseña les resultó poco útil.

•	text (cadena): Es el contenido de la reseña escrita por el usuario. Aquí se encuentra el texto completo de la reseña que describe la experiencia del usuario con el negocio.



In [ ]:
review.to_parquet(r'Dataset_filtrados\Total_reviews')


## tips

In [ ]:
#vemos datos en general
tips.head(1)

,user_id,business_id,text,date,compliment_count
8,VL12EhEdT4OWqGq0nIqkzw,xODBZmX4EmlVvbqtKN7YKg,Tacos,2012-07-27 01:48:24,0


In [ ]:
tips.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66363 entries, 8 to 908908
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   user_id           66363 non-null  object        
 1   business_id       66363 non-null  object        
 2   text              66363 non-null  object        
 3   date              66363 non-null  datetime64[ns]
 4   compliment_count  66363 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 3.0+ MB


In [ ]:
#visualizamos columnas
tips.columns

Index(['user_id', 'business_id', 'text', 'date', 'compliment_count'], dtype='object')

In [ ]:
# identificamos datos nulos
tips.isna().sum()

user_id             0
business_id         0
text                0
date                0
compliment_count    0
dtype: int64

In [ ]:
# observamos dimensiones del dataset
tips.shape

(66363, 5)

Las columna a eliminar es 'date’ ya que en el producto esa fecha no será relevante para la presentación de resultados. 

In [ ]:
tips.drop(columns= 'date', inplace = True)

In [ ]:
#verificamos resultado
tips.head(10)

,user_id,business_id,text,compliment_count
8,VL12EhEdT4OWqGq0nIqkzw,xODBZmX4EmlVvbqtKN7YKg,Tacos,0
13,MlnuJ7T14CE0JDK2ZIOx5g,MDr7KLYSPkEonvGojNEMBw,Let's go Yankees!,0
15,j2sEA3hiUcwHfq9Ml6M47g,EXYbKA1tocvOK_1tXxZXLA,Don't go for dinner. They close at 6. Really Y...,0
22,phKNHTaokisQkZ7qQbxryA,pWuoHHu0LuVrD0HjadDygg,Crepes please!!!,0
30,jDThlALkraoQLLBYHqY7FQ,9DJhhBqQSu-gTBwaqdhgpQ,Come early for the best service as they fill u...,0
41,XJDLaoN1PerKw2woiKeepA,t0zwddmbOGQOADrAxEPHQQ,Come by for some Jelly Bean sours this weekend!,0
43,nZB33t0RFtq_jTxJJJ6wfA,RNzhLU8zy1L5dBkjRK4xEA,Best steak tartare ever,0
57,9qZVikoJoqfQK9SBSynAGA,l-rGtJt0E7PAklT0IK7oFQ,Great atmosphere... Live music... Excellent dr...,0
69,VWJ8PSz6Sg5_AlBvQyGvpw,PcX8zGIJ_XHTAQAcEOwhLw,"Nice resort, right by the powder white beach!",0
77,2idS8cD2zvsLnV5A9IImGg,ihYtslFvge9B9KK1S_E8rw,Great good and service.,0


Al analizar la columna 'compliment_count', se observa que la gran mayoría de sus valores son 0. Solo hay alrededor de 1079 registros (aproximadamente un 1.6% del total) que contienen valores distintos de 0. Dado que esta proporción es bastante baja, hemos tomado la decisión de eliminar la columna 'compliment_count' de nuestros datos. Esto se debe a que los valores no son lo suficientemente significativos para influir en los resultados que estamos buscando presentar.

In [ ]:
#verificamos cantidad de datos distitnso a 0
(tips['compliment_count'] != 0).sum()

1079

In [ ]:
#eliinamos la columna 'compliment_count'
tips.drop(columns= 'compliment_count', inplace = True)
#verificamos resultado
tips.head()

,user_id,business_id,text
8,VL12EhEdT4OWqGq0nIqkzw,xODBZmX4EmlVvbqtKN7YKg,Tacos
13,MlnuJ7T14CE0JDK2ZIOx5g,MDr7KLYSPkEonvGojNEMBw,Let's go Yankees!
15,j2sEA3hiUcwHfq9Ml6M47g,EXYbKA1tocvOK_1tXxZXLA,Don't go for dinner. They close at 6. Really Y...
22,phKNHTaokisQkZ7qQbxryA,pWuoHHu0LuVrD0HjadDygg,Crepes please!!!
30,jDThlALkraoQLLBYHqY7FQ,9DJhhBqQSu-gTBwaqdhgpQ,Come early for the best service as they fill u...


### Diccionario de tips 


user_id: Es un identificador único para cada usuario en Yelp que ha dejado un consejo (tip) en un negocio.

business_id: Es un identificador único para el negocio en el que se dejó el consejo.

text: Es el contenido del consejo que el usuario dejó para el negocio. Es un comentario o sugerencia que el usuario comparte.

In [ ]:
tips.to_parquet(r'Dataset_filtrados\Total_tips')

## user

In [ ]:
#visualisamos datos de manera general
user.head(1)

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,average_stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,3.91,250,65,55,56,18,232,844,467,467,239,180


In [ ]:
#observamos columnas
user.columns

Index(['user_id', 'name', 'review_count', 'yelping_since', 'useful', 'funny',
       'cool', 'elite', 'friends', 'fans', 'average_stars', 'compliment_hot',
       'compliment_more', 'compliment_profile', 'compliment_cute',
       'compliment_list', 'compliment_note', 'compliment_plain',
       'compliment_cool', 'compliment_funny', 'compliment_writer',
       'compliment_photos'],
      dtype='object')

In [ ]:
# verificamos la existencia de datos nulos.
user.isna().sum()

user_id               0
name                  0
review_count          0
yelping_since         0
useful                0
funny                 0
cool                  0
elite                 0
friends               0
fans                  0
average_stars         0
compliment_hot        0
compliment_more       0
compliment_profile    0
compliment_cute       0
compliment_list       0
compliment_note       0
compliment_plain      0
compliment_cool       0
compliment_funny      0
compliment_writer     0
compliment_photos     0
dtype: int64

Después de un análisis exhaustivo de los datos contenidos en cada columna, hemos llegado a la conclusión de que las siguientes columnas serán eliminadas de nuestro conjunto de datos: 'yelping_since', 'compliment_hot', 'compliment_more', 'compliment_profile', 'compliment_cute', 'compliment_list', 'compliment_note', 'compliment_plain', 'compliment_cool', 'compliment_funny', 'compliment_writer' y 'compliment_photos'. Hemos tomado esta decisión debido a que, en el contexto de mostrar los usuarios con más reseñas en cada negocio o nicho, estas columnas no aportan relevancia significativa. 

In [ ]:
#establecemos una variable con las columnas a borrar
columnas_drop= ['yelping_since','elite', 'compliment_hot', 'compliment_more', 'compliment_profile', 'compliment_cute', 'compliment_list', 'compliment_note', 'compliment_plain', 'compliment_cool', 'compliment_funny', 'compliment_writer','compliment_photos']
#borramos columnas.
user.drop(columns= columnas_drop, inplace = True)
#verificamos resultados
user.head()

,user_id,name,review_count,useful,funny,cool,friends,fans,average_stars
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,7217,1259,5994,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,3.91
1,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,2086,1010,1003,"LuO3Bn4f3rlhyHIaNfTlnA, j9B4XdHUhDfTKVecyWQgyA...",52,3.32
2,q_QQ5kBBwlCcbL1s4NVK3g,Jane,1221,14953,9940,11211,"xBDpTUbai0DXrvxCe3X16Q, 7GPNBO496aecrjJfW6UWtg...",1357,3.85
3,xoZvMJPDW6Q9pDAXI0e_Ww,Ryan,535,1130,487,573,"6tbXpUIU6upoeqWNDo9k_A, Vlab9b73R5qPLIv6tE4DJA...",31,3.89
4,vVukUtqoLF5BvH_VtQFNoA,Charlene,37,63,3,27,"zkK6c9BcDyqreU0fqI_JLQ, opI1hhhFqElB6pptNH9ZqA...",4,4.51


### Diccionario de user 


•	user_id: Es un identificador único para el usuario. Es una cadena de texto que identifica al usuario en Yelp.

•	name: Es el nombre del usuario. Es una cadena de texto que representa el nombre del usuario en Yelp.

•	review_count: Es la cantidad total de reseñas que el usuario ha escrito en Yelp. Es un número entero que indica cuántas reseñas ha dejado el usuario.


•	useful: Representa la cantidad total de votos "útiles" que el usuario ha recibido en sus reseñas por parte de otros usuarios. Es un número entero que indica cuántos usuarios han encontrado útiles las reseñas del usuario.

•	funny: Representa la cantidad total de votos "graciosos" que el usuario ha recibido en sus reseñas por parte de otros usuarios. Es un número entero que indica cuántos usuarios han encontrado graciosas las reseñas del usuario.

•	cool: Representa la cantidad total de votos "frescos" que el usuario ha recibido en sus reseñas por parte de otros usuarios. Es un número entero que indica cuántos usuarios han encontrado frescas las reseñas del usuario.

•	friends: Indica la cantidad de amigos que el usuario tiene en Yelp. Puede estar en blanco (nulo) si el usuario no ha proporcionado esta información.

•	fans: Número de seguidores que el usuario tiene en Yelp.


In [ ]:
user.to_parquet(r'Dataset_filtrados\Total_user')
